# Web Scrapping with API

In [1]:
import config
import requests

In [2]:
# Create a list to store comment dictionaries
comments_list = []

In [3]:
company = ""
with open("company.txt","r") as file:
    company = file.read()

# Access Database

In [4]:
#!pip install pymysql

In [5]:
import connection

In [6]:
con = connection.connect()
stm = con.cursor()

In [7]:
q = f"select * from companydetails where CompanyName='{company}'"
stm.execute(q)

1

In [8]:
rows = stm.fetchall()

In [9]:
Reciver_Mail = rows[0][1]
page_id = rows[0][2]
FB_post_id = rows[0][3]
INS_post_id = rows[0][4]
prev = rows[0][5]

## (a) Facebook 
`Meta Graph API`

In [10]:
#Scrapping Facebook Comments
user_access_token = config.FB_user_access_token

url = f'https://graph.facebook.com/v18.0/{page_id}_{FB_post_id}/comments'

params = {
    'access_token': user_access_token,
    'fields': 'message,created_time',
}

response = requests.get(url, params=params)
data = response.json()

# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message': comment['message'],
                                    'Timestamp': comment['created_time'],
                                    'Source' : 'Facebook',
                              })
for comment in comments_list:
    print(comment)

## (b) Instagram 
`Meta Graph API`

In [11]:
#Scrapping Instagram comments
user_access_token = config.INS_user_access_token
url = f'https://graph.facebook.com/v18.0/{INS_post_id}/comments'
params = {
    'fields': 'text,timestamp',
    'access_token': user_access_token,
}

response = requests.get(url, params=params)
data = response.json()

# Populate the list with comments
if 'data' in data:
    for comment in data['data']:
        from_info = comment.get('from', {})
        comments_list.append({
                                    'Message':comment['text'],
                                    'Timestamp': comment['timestamp'],
                                    'Source' : 'Instagram',
                            })

for comment in comments_list:
    print(comment)

## Sentiment Analysis

## Roberta Pretrained Model

In [12]:
#!pip install matplotlib

In [13]:
#!pip install seaborn

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
#!pip install tqdm

In [16]:
from tqdm.notebook import tqdm

In [17]:
#pip install pywinpty

In [18]:
#!pip install transformers

In [19]:
#!pip install scipy

In [20]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [21]:
df = pd.DataFrame(comments_list, columns=['Message','Timestamp','Source'])

In [22]:
print(df.dtypes)
df

Message      object
Timestamp    object
Source       object
dtype: object


,Message,Timestamp,Source


In [23]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [24]:
print(df.dtypes)

Message              object
Timestamp    datetime64[ns]
Source               object
dtype: object


In [25]:
df.sort_values(by='Timestamp', inplace = True)

In [26]:
df

,Message,Timestamp,Source


In [27]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [28]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [29]:
#!pip install --upgrade jupyter

In [30]:
#!pip install --upgrade ipywidgets

In [31]:
base : float
base = 10.00

res = {
    'pos' : [],
    'neg' : []
}
for i,row in tqdm(df.iterrows(), total=len(df)):
    try:
        print(row);
        text = row['Message']
        #myid = row['Id']
        roberta_result = polarity_scores_roberta(text)
        print(roberta_result)
        if roberta_result['roberta_neg']>roberta_result['roberta_neu'] and roberta_result['roberta_neg']>roberta_result['roberta_pos'] :
            base-=roberta_result['roberta_neg']
            res['neg'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        if roberta_result['roberta_pos']>roberta_result['roberta_neu'] and roberta_result['roberta_pos']>roberta_result['roberta_neg'] :
            base+=roberta_result['roberta_pos']
            res['pos'].append({
                'Comment' : row['Message'],
                'Source' :row['Source']
            })
        #print(base);
    except RuntimeError:
        print(f'Broke for id {i}')

0it [00:00, ?it/s]

In [32]:
print(base)

10.0


# Update Database

In [33]:
q = f"update companydetails set Score = {base} where CompanyName = '{company}'"
stm.execute(q)
con.commit()

# Sending Email

In [34]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [35]:
def send_email(subject, body, to_email, smtp_server, smtp_port, smtp_username, smtp_password):
    # Create the MIME object
    msg = MIMEMultipart()
    msg['From'] = smtp_username
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)

    server.sendmail(smtp_username, to_email, msg.as_string())
    server.quit()

In [36]:
subject = "Change In Score Detected"
body = f"New Score : {base} \nDelta change : {base-prev} \nLast 5 negitive comments:{res['neg'][-5:]} \nLast 5 positive comments:{res['pos'][-5:]}"

In [37]:
smtp_server = "smtp.gmail.com"
smtp_port = 587 

In [38]:
if(prev!=base) :
    send_email(subject=subject,
            body=body,
            to_email=Reciver_Mail,
            smtp_server=smtp_server,
            smtp_port=smtp_port,
            smtp_username=config.Sender_mail,
            smtp_password=config.AppPassword)
    print("Mail send")

In [39]:
stm.close()
con.close()

In [40]:
text = f"{prev} {base}"

In [41]:
with open('score.txt','w') as file:
    file.write(text)